### Import Modules

In [9]:
# Import required modules
import os
import sys
import re
import argparse
from Bio import SeqIO
import numpy as np
import pandas as pd
import polars as pl

### Function Definitions

In [ ]:
# Function #1: fasta_to_dataframe
def fasta_to_dataframe(fasta):
	"""
	Converts a FASTA file to a polars DataFrame object. 

	Args:
		fasta (str): Path to the FASTA file.

	Returns:
		pl.DataFrame: A DataFrame with 'header' and 'sequence' columns.
	"""

	# Parse the FASTA and extract headers and sequences
	records = SeqIO.parse(fasta, 'fasta')
	# Initialize empty lists to store headers and sequences
	headers = []
	sequences = []

	# Iterate over the records and append the headers and sequnences to the respective lists
	for record in records:
		headers.append(str(record.id))
		sequences.append(str(record.seq))
	
	# Create a polars data frame from the headers and sequences
	df = pl.DataFrame({'headers': headers, 'sequences': sequences})

	return df

# Function #2: find_subsequence_range
def find_subsequence_range(sequence):
	"""
	Find the start and end positions of a subsequence within a larger sequence.

	Args:
		sequence (str): The sequence to search within.
	
	Returns:
		List of tuples: List of start and end positions of the subsequence 
		where N's are found. Each tuple is in the form (start, end), where 
		start is zero indexed like a BED file.
	"""

	# Initialize an empty variable to store the current sequence coordinates
	coordinates = []
	# Initialize a variable to store the start position of the subsequence
	start = None

	# Iterate over the sequence and find the start and end positions of the subsequence
	for coor in range(len(sequence)):
		# If the current base is an 'N' and the start position is not set, set the start position
		if sequence[coor] == 'N' and start is None:
			start = coor
		# If the current base is not an 'N' and the start position is set, set the end position
		elif sequence[coor] != 'N' and start is not None:
			coordinates.append((start, coor - 1))
			start = None

	# If the start position is set at the end of the sequence, set the end position
	if start is not None:
		coordinates.append((start, len(sequence) - 1))

	return coordinates

In [27]:
# Test function #2
sequence = 'asdfghjklNNNNNfafafafafafjakfjafkfjkfajkjfNNNNNNNNNNkjfkjafk'
coordinates = find_subsequence_range(sequence)
print(coordinates)

[(9, 13), (42, 51)]


### Read in FASTA

In [15]:
# Set the path to the input FASTA file
fasta_file = '/home/administrator/ExtraSSD2/Kaas/Projects/SquamateAlignments/Reference_Genomes/Sekar_Genomes/Scaffold_Assemblies/Elapidae/Naja/Naja_nigricollis_najNig1/Assembly/najNig2.ragtag.scaffold_naNa.REHEADER.MT.fasta'
os.path.exists(fasta_file)

True

### Convert FASTA file to polars DataFrame

In [16]:
# Convert the fasta file to a polars DataFrame
df = fasta_to_dataframe(fasta_file)

In [17]:
df

headers,sequences
str,str
"""MT""","""GTTGTCATAGCTTACCTATCAAGCATAGCA…"
"""scaffoldma1""","""CAGCTGTTCCAACAATCAGCTGTGCCGGNN…"
"""scaffoldma2""","""CTGGGAGGGGAGGCAGGTATTTCCAGTGAT…"
"""scaffoldma3""","""CCCCTCCCTTCCAGCACTGATAATGTTATC…"
"""scaffoldma4""","""AGACACCAGTCCTCTATTTTAATTACAAAT…"
…,…
"""scaffoldun1024""","""GGTTTTCAGGCTTAAGGTGGGATTGGAACT…"
"""scaffoldun1025""","""CCGAACCTTCCTTAGGACTGAAGCTTCTAT…"
"""scaffoldun1026""","""ATTGTAGCAGATAATTTCATGTACTATGCT…"


### Alter the data frame

In [ ]:
df2 = (
	df
	# Filter the DataFrame to only the one chrom to reduce complexity
	.filter(pl.col('headers') == "scaffoldma1")
	.with_columns(
		pl.col('sequences').map_elements(lambda x: find_subsequence_range(x)).alias('coordinates')
	)
	# Drop the sequences column, as it is no longer needed
	.drop(['sequences'])
	# Explode the coordinates column to create a new row for each coordinate
	.explode('coordinates')
	# Create two new columns for the start and end cooordinates
	.with_columns(
		pl.col('coordinates').list.get(0).alias('start'),
		pl.col('coordinates').list.get(1).alias('end')
	)
	# Drop the coordinates column, as it is no longer needed
	.drop(['coordinates'])
)

/tmp/ipykernel_1456097/3772271412.py:5: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns(


In [40]:
df2

headers,coordinates,start,end
str,list[i64],i64,i64
"""scaffoldma1""","[28, 127]",28,127
"""scaffoldma1""","[35185, 35284]",35185,35284
"""scaffoldma1""","[127631, 127730]",127631,127730
"""scaffoldma1""","[212420, 212519]",212420,212519
"""scaffoldma1""","[217359, 217458]",217359,217458
…,…,…,…
"""scaffoldma1""","[382621344, 382621443]",382621344,382621443
"""scaffoldma1""","[382661103, 382661202]",382661103,382661202
"""scaffoldma1""","[382693490, 382693589]",382693490,382693589
